##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Primeiros passos do TensorFlow 2 para iniciantes

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a> </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a> </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a> </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a> </td>
</table>

Esta breve introdução usa o [Keras](https://www.tensorflow.org/guide/keras/overview) para:

1. Carregar um dataset pré-criado.
2. Criar um modelo de aprendizado de máquina de rede neural que classifica imagens.
3. Treinar essa rede neural.
4. Avaliar a exatidão do modelo.

Este é um notebook do [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb). Os programas em Python são executados diretamente no navegador — uma ótima maneira de aprender e usar o TensorFlow. Para seguir este tutorial, execute o notebook no Google Colab clicando no botão no topo desta página.

1. No Colab, conecte a um runtime do Python: no canto superior direito da barra de menu, selecione *CONNECT* (CONECTAR).
2. Para executar todo o código no notebook, selecione **Runtime** &gt; **Run all** (Executar tudo). Para executar uma célula de código de cada vez, passe o cursor sobre cada célula e selecione o ícone **Run cell** (Executar célula).

![Ícone para executar célula](https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/tutorials/quickstart/images/beginner/run_cell_icon.png?raw=1)

## Configure o TensorFlow

Para começar, importe o TensorFlow para seu programa:

In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.19.0


Se você estiver acompanhando no seu próprio ambiente de desenvolvimento em vez do [Colab](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb), veja o [guia de instalação](https://www.tensorflow.org/install) para configurar o TensorFlow para desenvolvimento.

Observação: confira se você atualizou para o `pip` mais recente antes de instalar o pacote do TensorFlow 2 caso esteja usando seu próprio ambiente de desenvolvimento. Confira mais detalhes no [guia de instalação](https://www.tensorflow.org/install).

## Carregue um dataset

Carregue e prepare o [dataset MNIST](http://yann.lecun.com/exdb/mnist/). Os valores de pixel das imagens vão de 0 a 255. Ajuste esses valores para um intervalo de 0 a 1 ao dividir os valores por `255.0`. Isso também converte os dados de amostra de números inteiros para números em ponto flutuante:

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


## Crie um modelo de aprendizado de máquina

Crie um modelo `tf.keras.Sequential`:

In [3]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


[`Sequential`](https://www.tensorflow.org/guide/keras/sequential_model) é útil para empilhar camadas quando cada uma tem um [tensor](https://www.tensorflow.org/guide/tensor) de entrada e um tensor de saída. As camadas são funções com uma estrutura matemática conhecida que podem ser reutilizadas e têm variáveis que podem ser treinadas. A maioria dos modelos do TensorFlow são compostos de camadas. Esse modelo usa as camadas [`Flatten`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten), [`Dense`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense) e [`Dropout`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout).

Para cada exemplo, o modelo retorna um vetor de [logits](https://developers.google.com/machine-learning/glossary#logits) ou pontuações [log-odds](https://developers.google.com/machine-learning/glossary#log-odds), uma para cada classe.

In [4]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.07664758,  0.67413926,  1.0413301 ,  0.0807932 ,  0.40247634,
        -0.27727848,  0.11417972,  0.401039  ,  0.1420201 ,  0.5428669 ]],
      dtype=float32)

A função `tf.nn.softmax` converte esses logits em *probabilidades* para cada classe:

In [5]:
tf.nn.softmax(predictions).numpy()

array([[0.06367096, 0.13489753, 0.19474797, 0.07452758, 0.1028069 ,
        0.0520965 , 0.07705781, 0.10265924, 0.07923327, 0.11830228]],
      dtype=float32)

Observação: é possível fazer o bake da função `tf.nn.softmax` na função de ativação para a última camada da rede. Isso pode fazer com que a saída do modelo seja mais diretamente interpretável, mas essa abordagem não é recomendada, já que é impossível fornecer um cálculo de perda numericamente estável e exato para todos os modelos ao usar uma saída de softmax.

Defina uma função de perda para treinamento usando `losses.SparseCategoricalCrossentropy`:

In [6]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

A função de perda obtém um vetor de valores verdadeiros e um vetor de logits e retorna uma perda escalar para cada exemplo. Essa perda é igual à probabilidade log negativa da classe verdadeira: a perda é zero se o modelo tem certeza da classe correta.

Esse modelo não treinado fornece probabilidades perto de serem aleatórias (1/10 para cada classe), então a perda inicial deve ser próxima de `-tf.math.log(1/10) ~= 2.3`.

In [7]:
loss_fn(y_train[:1], predictions).numpy()

np.float32(2.9546576)

Antes de começar a treinar, configure e compile o modelo usando o `Model.compile` do Keras. Defina a classe [`optimizer`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) (otimizador) como `adam`, defina o `loss` (perda) como a função `loss_fn` definida antes e especifique uma métrica para ser avaliada para o modelo ao configurar o parâmetro `metrics` (métricas) como `accuracy` (exatidão).

In [8]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## Treine e avalie seu modelo

Use o método `Model.fit` para ajustar os parâmetros do seu modelo e reduzir a perda:

In [9]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.8542 - loss: 0.4954
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.9533 - loss: 0.1606
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.9667 - loss: 0.1110
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9730 - loss: 0.0882
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.9772 - loss: 0.0745


O método `Model.evaluate` verifica o desempenho de um modelo, geralmente em um [dataset de validação](https://developers.google.com/machine-learning/glossary#validation-set) ou de [teste](https://developers.google.com/machine-learning/glossary#test-set).

In [10]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - 2ms/step - accuracy: 0.9735 - loss: 0.0824


[0.08241119235754013, 0.9735000133514404]

O classificador de imagens já está treinado com cerca de 98% de exatidão nesse dataset. Para saber mais, leia os [tutoriais do TensorFlow](https://www.tensorflow.org/tutorials/).

Se você quiser que seu modelo retorne uma probabilidade, empacote o modelo treinado e anexe o softmax a ele:

In [11]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [12]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[7.25045624e-08, 1.95577377e-09, 4.87170155e-06, 4.00214485e-05,
        5.13693124e-12, 6.50496247e-07, 2.71641542e-13, 9.99949217e-01,
        6.61825723e-08, 5.17556828e-06],
       [4.98869701e-09, 5.26275689e-05, 9.99946237e-01, 4.74602814e-07,
        2.21378668e-14, 2.05184108e-07, 3.84446992e-07, 9.28827848e-12,
        1.27700233e-07, 1.73713930e-12],
       [1.01684456e-07, 9.99785841e-01, 1.31877550e-05, 2.45045641e-07,
        6.18798276e-06, 1.92478637e-06, 3.73770627e-06, 1.85997284e-04,
        2.70135547e-06, 6.94921241e-08],
       [9.99976873e-01, 1.13946665e-10, 1.62450906e-05, 1.46027865e-08,
        1.54889956e-09, 3.59313873e-07, 2.48420861e-07, 4.18105230e-07,
        1.62577756e-08, 5.82715575e-06],
       [3.92267384e-06, 6.98227920e-10, 1.37689940e-05, 6.47098659e-08,
        9.96863604e-01, 1.22746746e-06, 1.18368671e-05, 1.26878047e-04,
        1.83246539e-06, 2.97693769e-03]], dtype=float32)>

## Conclusão

Parabéns! Você treinou um modelo de aprendizado de máquina usando um dataset pré-criado com a API [Keras](https://www.tensorflow.org/guide/keras/overview).

Para obter mais exemplos de uso do Keras, confira os [tutoriais](https://www.tensorflow.org/tutorials/keras/). Para saber mais sobre como criar modelos com o Keras, leia os [guias](https://www.tensorflow.org/guide/keras). Se quiser saber mais sobre como carregar e preparar dados, consulte os tutoriais sobre [carregamento de dados de imagem](https://www.tensorflow.org/tutorials/load_data/images) ou [carregamento de dados CSV](https://www.tensorflow.org/tutorials/load_data/csv).
